In [1]:
import sys;sys.path.append('../../')
import os

from rich import print

os.environ['OPENAI_API_KEY'] = open('../../../openai.key').read().strip()

In [2]:
import csv
import random
from typing import List

from multi_agent_llm import OpenAILLM
from multi_agent_llm.agents.mlu.lu import CLU
from pydantic import BaseModel, Field

In [3]:
class GPQAQuestion(BaseModel):
    question: str
    options: List[str]
    correct_answer: str

class QA(BaseModel):
    explanation: str = Field(..., description="Explanation for the answer")
    answer: str = Field(..., description="Final answer (A/B/C/D/Uncertain) without any additional explanation.")

# GPQADataset class to load the dataset from a CSV file
class GPQADataset:
    def __init__(self, file_path: str):
        self.questions = self._load_questions(file_path)

    def _load_questions(self, file_path: str) -> List[GPQAQuestion]:
        questions = []
        with open(file_path, "r", encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                options = [
                    row["Correct Answer"],
                    row["Incorrect Answer 1"],
                    row["Incorrect Answer 2"],
                    row["Incorrect Answer 3"],
                ]
                question = GPQAQuestion(
                    question=row["Question"],
                    options=options,
                    correct_answer=row["Correct Answer"]
                )
                questions.append(question)
        return questions

    def __len__(self) -> int:
        return len(self.questions)

    def __getitem__(self, index: int) -> GPQAQuestion:
        return self.questions[index]

In [4]:
def train_and_test_mlu(
    clu: CLU,
    dataset: GPQADataset,
    train_indices: List[int],
    test_indices: List[int]
):
    def shuffle_options_and_set_answer(question_data: GPQAQuestion):
        """Randomize the options and determine the new correct answer label."""
        options = question_data.options.copy()
        correct_answer = question_data.correct_answer

        # Shuffle the options
        random.shuffle(options)

        # Determine the new label for the correct answer
        correct_label = chr(options.index(correct_answer) + ord('A'))

        return options, correct_label

    def format_options(options: List[str]) -> str:
        """Format the options into labeled form (A. option, B. option, etc.)"""
        labeled_options = [
            f"{chr(65 + i)}. {option}" for i, option in enumerate(options)
        ]
        return "\n".join(labeled_options)

    # Training Phase
    print("\n=== Training Phase ===")
    for idx in train_indices:
        question_data = dataset[idx]
        task = f"Question: {question_data.question}\n"

        # Shuffle options and set the correct label
        shuffled_options, correct_label = shuffle_options_and_set_answer(question_data)
        task += f"Options:\n{format_options(shuffled_options)}\n\nPick the correct option after thinking step by step."

        # Prepare the expected output with the shuffled correct answer
        expected_output = QA(
            explanation=f"The correct answer is {correct_label} because of {question_data.correct_answer}.",
            answer=correct_label
        )

        print(f"Training on index {idx}: {task}")
        clu.training(task=task, expected_output=expected_output)

    # Testing Phase (Inference)
    print("\n=== Testing Phase (Inference) ===")
    for idx in test_indices:
        question_data = dataset[idx]
        task = f"Question: {question_data.question}\n"

        # Shuffle options and set the correct label
        shuffled_options, correct_label = shuffle_options_and_set_answer(question_data)
        task += f"Options:\n{format_options(shuffled_options)}\n\nPick the correct option after thinking step by step."

        # Perform inference
        inferred_output = clu.inference(task=task, output_schema=QA)

        print(f"Inference on index {idx}: {task}")
        print(f"Inferred Answer: {inferred_output.operational_agent_response.response.answer}")
        print(f"Correct Answer: {correct_label}")
        print()



In [5]:
main_role="""Your role is to engage in meta-learning to master the process of solving complex scientific reasoning tasks by learning from examples. You will observe and analyze various scientific questions, along with their reasoning and correct answers, to extract the underlying patterns and strategies required to solve similar tasks in the future. The core of your approach should focus on:

Learning from Examples: Study the provided examples of scientific reasoning, including both the questions and their step-by-step solutions, to internalize the key principles and problem-solving approaches.
Identifying Patterns: Recognize the recurring structures, logical steps, and scientific principles that frequently appear in questions, enabling you to generalize solutions to new problems.
Meta-Cognition: Develop an understanding of how to reason effectively by reflecting on the thought processes used to arrive at correct answers, and adapt these processes to a wide range of scientific questions.
Applying Adaptive Reasoning: Use the patterns and strategies learned from prior examples to approach new questions with flexibility, applying appropriate reasoning steps based on the unique characteristics of each task.
Self-Improvement: Continuously improve by comparing your reasoning and answers to provided solutions, identifying areas where your reasoning can be refined or extended, and updating your approach to improve performance on future tasks."""

In [6]:
llm = OpenAILLM(model_name="gpt-4o-mini")
clu = CLU(llm=llm, main_role=main_role, collection_name="gpqa-dataset")

# Load the dataset
dataset = GPQADataset(file_path="./gpqa_diamond.csv")

In [7]:
train_indices = [0, 1, 2, 3, 4]  # Example indices for training
test_indices = [5, 6, 7, 8, 9]  # Example indices for testing

# Run the training and testing
train_and_test_mlu(clu, dataset, train_indices, test_indices)

=== Training Phase ===

Training on index 0: Question: Two quantum states with energies E1 and E2 have a lifetime of 10^-9 sec and 10^-8 
sec, respectively. We want to clearly distinguish these two energy levels. Which one of the following options could
be their energy difference so that they can be clearly resolved?

Options:
A. 10^-8 eV

B. 10^-11 eV
C. 10^-4 eV
D. 10^-9 eV

Pick the correct option after thinking step by step.

                                                 Inference Result                                                  
╭────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────╮
│          Field │ Value                                                                                          │
├────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────┤
│       Response │ The correct option is A. 10^-8 eV, as it is an energy difference that exceeds the              │
│                │ uncertainties calculated from the lifetimes of the two quantum states, allowing for clear      │
│                │ distinction between them.                                                                      │
│      Reasoning │ To resolve the two quantum states with lifetimes of 10^-9 sec and 10^-8 sec using the          │
│                │ Energy-Time Uncertainty Principle, we need to calculate the energy uncertainties associated    │
│                │ with each state. The principle states that ΔEΔt ≥ ℏ/2, where ℏ (reduced Planck's constant) is  │
│                │ approximately 1.05 × 10^-34 Js.                                                                │
│                │                                                                                                │
│                │ 1. For the state with a lifetime of 10^-9 seconds:                                             │
│                │    ΔE1 ≈ ℏ / Δt1 = (1.05 × 10^-34 J·s) / (10^-9 s) = 1.05 × 10^-25 J ≈ 6.58 × 10^-26 eV.       │
│                │                                                                                                │
│                │ 2. For the state with a lifetime of 10^-8 seconds:                                             │
│                │    ΔE2 ≈ ℏ / Δt2 = (1.05 × 10^-34 J·s) / (10^-8 s) = 1.05 × 10^-26 J ≈ 6.58 × 10^-27 eV.       │
│                │                                                                                                │
│                │ Now that we have the uncertainties:                                                            │
│                │ - ΔE1 ≈ 6.58 × 10^-26 eV                                                                       │
│                │ - ΔE2 ≈ 6.58 × 10^-27 eV                                                                       │
│                │                                                                                                │
│                │ To clearly distinguish these states, we require an energy difference that exceeds both         │
│                │ uncertainties. The minimum required energy difference should be greater than the largest       │
│                │ uncertainty:                                                                                   │
│                │ **Minimum Energy Difference > ΔE1 ≈ 6.58 × 10^-26 eV.**                                        │
│                │                                                                                                │
│                │ Now, evaluating the options:                                                                   │
│                │ A. 10^-8 eV = 1 × 10^-8 eV (this exceeds the uncertainty)                                      │
│                │ B. 10^-11 eV (this is less than the uncertainties)                                             │
│                │ C. 10^-4 eV (this far exceeds the uncertainty)                                                 │
│                │ D. 10^-9 eV (this exceeds the uncertainty)                                                     │
│                │                                                                                                │
│                │ The lowest viable option is 10^-8 eV (option A), which is sufficient to distinguish the two    │
│                │ energies, while the option clearly ab

Training on index 1: Question: trans-cinnamaldehyde was treated with methylmagnesium bromide, forming product 1.

1 was treated with pyridinium chlorochromate, forming product 2.

3 was treated with (dimethyl(oxo)-l6-sulfaneylidene)methane in DMSO at elevated temperature, forming product 3.

how many carbon atoms are there in product 3?
Options:
A. 10
B. 12
C. 11
D. 14

Pick the correct option after thinking step by step.

                                                 Inference Result                                                  
╭────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────╮
│          Field │ Value                                                                                          │
├────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────┤
│       Response │ B. 12                                                                                          │
│      Reasoning │ 1. We start with trans-cinnamaldehyde, which has 9 carbon atoms (C9). \n2. When treated with   │
│                │ methylmagnesium bromide (a Grignard reagent), the carbon number increases due to the addition  │
│                │ of a methyl group. So, product 1 contains C9 + C1 = C10 (10 carbon atoms). \n3. Product 1      │
│                │ (C10) is then treated with pyridinium chlorochromate, which typically involves oxidation but   │
│                │ does not change the carbon count. Therefore, product 2 also contains C10 (still 10 carbon      │
│                │ atoms). \n4. Product 2 is treated with (dimethyl(oxo)-l6-sulfaneylidene)methane in DMSO at     │
│                │ elevated temperatures. This reaction involves the addition of carbon, likely from the reagent  │
│                │ being used. The structure of the reagent suggests that it contributes 2 additional carbon      │
│                │ atoms. Thus, product 3 will have C10 + C2 = C12 (12 carbon atoms).\n5. Therefore, the total    │
│                │ number of carbon atoms in product 3 is 12.                                                     │
│     Confidence │ 0.95                                                                                           │
│ Knowledge Used │ Grasp fundamental organic chemistry concepts, especially reaction mechanisms, to improve       │
│                │ reasoning in chemical processes and scientific inquiries.                                      │
│                │ tags: Organic Chemistry,Reaction Mechanisms,Scientific Reasoning,Chemical                      │
│                │ Processes,Step-by-Step Thinking In the reaction sequence, trans-cinnamaldehyde undergoes       │
│                │ transformations leading to various products. When treated with methylmagnesium bromide,        │
│                │ product 1 is formed, which is then oxidized by pyridinium chlorochromate to yield product 2.   │
│                │ Product 3 results from treating product 2 in DMSO with a specific reagent at elevated          │
│                │ temperature. To find the carbon count in product 3, we trace the carbons through these         │
│                │ reactions. The final count is 11 carbon atoms, corresponding to option 'C'. Future prompts     │
│                │ should clarify answers and directly link them to option labels for improved clarity.           │
│                │ tags: science,chemistry,carbon count,reaction sequence,effective communication,prompt          │
│                │ improvement A systematic approach to analyzing organic reactions involves first identifying    │
│                │ functional groups, then tracking formations and transformations. This structured reasoning     │
│                │ enhances understanding and problem-solving skills in chemistry.                                │
│                │ tags: Step-by-step reasoning,Organic chemistry,Functional groups,Chemical                      │
│                │ transformations,Scientific analysis Understanding common organic reagents and their influence  │
│                │ on molecular structures enhances reasoning and prediction accuracy in chemical reactions. This │
│                │ knowledge allows for better analysis and interpretation of chemical processes.                 │
│                │ tags: organic chemistry,chemical reac

...

Correct Answer: D

                                                 Inference Result                                                  
╭────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────╮
│          Field │ Value                                                                                          │
├────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────┤
│       Response │ explanation='The density of Option b is given as 5.5 g/cm³, which is significantly higher than │
│                │ the calculated densities of the other options. Therefore, Option b has the highest density of  │
│                │ all the provided exoplanet options.' answer='D'                                                │
│      Reasoning │ To evaluate which exoplanet has the highest density, we can calculate the density for each     │
│                │ option based on the mass and volume relationships. The density formula is: Density = Mass /    │
│                │ Volume, where the volume of a sphere (like a planet) is calculated as Volume =                 │
│                │ (4/3)π(Radius)^3. For comparative purposes, we'll start with Earth's values: mass = 1 Earth    │
│                │ mass and radius = 1 Earth radius.                                                              │
│                │                                                                                                │
│                │ 1. **Option a**: Earth-mass (1) and Earth-radius (1).                                          │
│                │    - Mass = 1 Earth mass                                                                       │
│                │    - Volume = (4/3)π(1)^3 ≈ 4.19  (in Earth units)                                             │
│                │    - Density = 1 / 4.19 ≈ 0.239 g/cm³.                                                         │
│                │                                                                                                │
│                │ 2. **Option b**: 2 Earth masses and a density of approximately 5.5 g/cm³.                      │
│                │    - Given density = 5.5 g/cm³ directly.                                                       │
│                │                                                                                                │
│                │ 3. **Option c**: 5 times more massive than Earth but the same composition and radius.          │
│                │    - Mass = 5 Earth masses.                                                                    │
│                │    - Radius = 1 Earth radius (since composition) implies it's still the same size; hence       │
│                │ volume = (4/3)π(1)^3.                                                                          │
│                │    - Density = 5 / 4.19 ≈ 1.194 g/cm³.                                                         │
│                │                                                                                                │
│                │ 4. **Option d**: Half the mass of Earth, same composition.                                     │
│                │    - Mass = 0.5 Earth masses.                                                                  │
│                │    - Volume = (4/3)π(1)^3.                                                                     │
│                │    - Density = 0.5 / 4.19 ≈ 0.119 g/cm³.                                                       │
│                │                                                                                                │
│                │ Now comparing densities:                                                                       │
│                │ - Option a: 0.239 g/cm³                                                                        │
│                │ - Option b: 5.5 g/cm³                

Inference on index 9: Question: Among the following exoplanets, which one has the highest density?

a) An Earth-mass and Earth-radius planet.
b) A planet with 2 Earth masses and a density of approximately 5.5 g/cm^3.
c) A planet with the same composition as Earth but 5 times more massive than Earth.
d) A planet with the same composition as Earth but half the mass of Earth.
Options:
A. c
B. a
C. d
D. b

Pick the correct option after thinking step by step.

Inferred Answer: D

Correct Answer: A

In [8]:
clu.print_knowledge_base()

General Knowledge Base:

Entry 1: Explicit formatting of responses enhances clarity. Always specify the selected answer option and include 
reasoning to improve coherence in communication. This structured approach aids in reasoning and answering 
scientific questions effectively.
tags: clarity,coherence,structured responses,scientific reasoning,communication skills

Entry 2: To enhance clarity in scientific reasoning, maintain a logical flow in communication. This minimizes 
confusion and ensures the audience understands the concepts being explained.
tags: scientific reasoning,logical flow,effective communication,clarity,minimize confusion

Entry 3: This knowledge is crucial for academic environments, scientific assessments, and situations that demand 
precise, structured answers, particularly in multiple-choice questions.
tags: Academic Settings,Scientific Exams,Structured Responses,Multiple-Choice Format,Clear Communication

Entry 4: Regularly reviewing fundamental organic chemistry principles is essential for ensuring a strong 
foundation, which aids in addressing more complex problems effectively.
tags: Organic Chemistry,Fundamental Principles,Problem-Solving,Review Strategies,Complex Problem Resolution

...

Entry 124: Maxwell's equations describe the behavior of electric and magnetic fields. If magnetic monopoles were 
introduced, the equations would be expanded to include additional terms. Specifically, the Gauss's law for 
magnetism would change to include a monopole source term, leading to a formulation that includes both electric and 
magnetic charges. Each equation's structure would reflect symmetry between electric and magnetic fields. 
Understanding these changes helps in grasping fundamental concepts in electromagnetism and theoretical physics.
tags: Maxwell's Equations,Magnetic Monopoles,Electromagnetism,Theoretical Physics,Charge Symmetry

Entry 125: Visual aids, such as reaction schemes, help track atomic changes in complex organic reactions, improving
understanding of these processes.
tags: visual aids,reaction schemes,organic chemistry,atom tracking,comprehension enhancement

Entry 126: To improve transparency and comprehension in calculations, follow these steps: 1. Define the problem 
clearly, indicating what is to be solved. 2. Identify relevant formulas or concepts needed for the calculations. 3.
Break the calculation into smaller, manageable steps, explaining the reasoning at each step. 4. Perform the 
calculations systematically, showing all intermediary results. 5. Conclude with a summary of the findings, relating
them back to the original problem. This structured approach enhances understanding and allows for easier 
verification of work.
tags: calculation transparency,step-by-step reasoning,scientific problem solving,mathematical 
comprehension,educational strategies

Entry 127: To respond effectively to scientific questions, state the final answer clearly at the beginning. Then, 
follow up with detailed reasoning to support the conclusion. This method enhances clarity and understanding of the 
reasoning process.
tags: Scientific Reasoning,Effective Communication,Step-by-Step Explanation,Conclusion First,Clarity in Science

Entry 128: Utilize a structured reasoning framework in discussions by clearly outlining premises that lead to 
conclusions. This approach ensures comprehensive coverage of scientific principles, promoting critical thinking and
systematic analysis in problem-solving.
tags: structured reasoning,scientific principles,critical thinking,systematic analysis,comprehensive coverage